In [1]:
import pandas as pd
import numpy as np

In [2]:
movies_df = pd.read_csv("Movie.csv",sep=';')
movies_df.head(11)

,movie_id,title,language,genres,year,link,content,movie_image_file,date_posted,user_id
0,1,3 Idiots,Hindi,"Comedy,Drama",2009,K0eDlFX9GMc,"In college, Farhan and Raju form a great bond ...",0089db3a8f56c033.png,2020-05-24 06:05:05.373129,1
1,2,Mission Mangal,Hindi,"Drama,Sci-fi",2019,q10nfS9V090,A group of scientists at ISRO battle in their ...,5dcca46018a09d86.jpg,2020-05-24 17:11:19.421290,1
2,3,Life of Pi,Hindi,"Adventure,Drama",2012,j9Hjrs6WQ8M,Pi Patel finds a way to survive in a lifeboat ...,957ace09ef5968ba.jpg,2020-05-24 17:52:03.704763,1
3,4,Uri: The Surgical Strike,Hindi,"Action,War",2019,VVY3do673Zc,Major Vihaan Singh Shergill of the Indian Army...,aac4bae0ab0d5c74.jpg,2020-05-24 18:19:49.340209,1
4,5,Kesari,Hindi,"Action,War",2019,JFP24D15_XM,"Havildar Ishar Singh, a soldier in the British...",a89e150b9dfdec7e.jpg,2020-05-24 18:21:38.391690,1
5,6,Parmanu: The Story of Pokhran,Hindi,"Action,Drama",2018,qN_9DnBh3hM,DescriptionIAS officer Ashwat is tasked to lea...,531b33b4d5e67934.jpg,2020-05-25 02:33:36.841468,1
6,7,Guardians of the Galaxy,Bengali,"Adventure,Sci-fi",2014,Sskn2k8G_so,Peter escapes from the planet Morag with a val...,aa93f38bf9287ecc.jpg,2020-05-25 02:37:45.765751,1
7,8,Guardians of the Galaxy Vol. 2,English,"Adventure,Sci-fi",2017,pr7tDrwQ3t8,"After a successful mission, Quill and his team...",cd1549cb6d98605e.jpg,2020-05-25 02:58:07.314884,1
8,9,Avengers: Endgame,Bengali,"Action,Sci-fi",2019,TcMBFSGVi1c,"After Thanos, an intergalactic warlord, disint...",5bdc3f15cb66eb0d.jpg,2020-05-25 03:02:24.256598,1
9,10,Captain America: The First Avenger,English,"Action,Adventure",2011,JerVrbLldXw,"During World War II, Steve Rogers decides to v...",eb2c84c18eab5ec0.png,2020-05-25 03:05:06.182982,1


In [3]:
user_df = pd.read_csv("Credits.csv",sep=';')
user_df.head(11)

,movie_id
0,13
1,3
2,4
3,1
4,5
5,2
6,2
7,2
8,5
9,2


In [4]:
user_df.drop_duplicates(subset = 'movie_id',inplace = True)
user_df.reset_index(drop=True, inplace=True)

In [5]:
movies_df_merge = movies_df.merge(user_df, on='movie_id')
movies_df_merge.head(10)

,movie_id,title,language,genres,year,link,content,movie_image_file,date_posted,user_id
0,1,3 Idiots,Hindi,"Comedy,Drama",2009,K0eDlFX9GMc,"In college, Farhan and Raju form a great bond ...",0089db3a8f56c033.png,2020-05-24 06:05:05.373129,1
1,2,Mission Mangal,Hindi,"Drama,Sci-fi",2019,q10nfS9V090,A group of scientists at ISRO battle in their ...,5dcca46018a09d86.jpg,2020-05-24 17:11:19.421290,1
2,3,Life of Pi,Hindi,"Adventure,Drama",2012,j9Hjrs6WQ8M,Pi Patel finds a way to survive in a lifeboat ...,957ace09ef5968ba.jpg,2020-05-24 17:52:03.704763,1
3,4,Uri: The Surgical Strike,Hindi,"Action,War",2019,VVY3do673Zc,Major Vihaan Singh Shergill of the Indian Army...,aac4bae0ab0d5c74.jpg,2020-05-24 18:19:49.340209,1
4,5,Kesari,Hindi,"Action,War",2019,JFP24D15_XM,"Havildar Ishar Singh, a soldier in the British...",a89e150b9dfdec7e.jpg,2020-05-24 18:21:38.391690,1
5,8,Guardians of the Galaxy Vol. 2,English,"Adventure,Sci-fi",2017,pr7tDrwQ3t8,"After a successful mission, Quill and his team...",cd1549cb6d98605e.jpg,2020-05-25 02:58:07.314884,1
6,9,Avengers: Endgame,Bengali,"Action,Sci-fi",2019,TcMBFSGVi1c,"After Thanos, an intergalactic warlord, disint...",5bdc3f15cb66eb0d.jpg,2020-05-25 03:02:24.256598,1
7,10,Captain America: The First Avenger,English,"Action,Adventure",2011,JerVrbLldXw,"During World War II, Steve Rogers decides to v...",eb2c84c18eab5ec0.png,2020-05-25 03:05:06.182982,1
8,11,Deadpool 2,Bengali,"Action,Comedy",2018,D86RtevtfrA,Deadpool protects a young mutant Russell from ...,cd348927eb33511e.jpg,2020-05-25 03:11:23.059491,1
9,12,Ant-Man,English,"Action,Adventure",2015,pWdKf3MneyI,"Scott, a master thief, gains the ability to sh...",cf21557e643b19f2.jpg,2020-05-25 03:15:25.455144,1


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

In [7]:
# Fitting the TF-IDF on the 'overview' text
tfv_matrix = tfv.fit_transform(movies_df_merge['genres'])

In [8]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [9]:
# Reverse mapping of indices and movie titles
indices = pd.Series(movies_df_merge.index, index=movies_df_merge['movie_id'])
indices

movie_id
1      0
2      1
3      2
4      3
5      4
8      5
9      6
10     7
11     8
12     9
13    10
17    11
dtype: int64

In [10]:
def give_rec(movie_id, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[movie_id]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return movies_df_merge['movie_id'].iloc[movie_indices]

In [12]:
give_rec(13)

10    13
5      8
1      2
3      4
4      5
8     11
11    17
7     10
9     12
0      1
2      3
Name: movie_id, dtype: int64